# Importing libraries and defining functions

In [ ]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [ ]:
def Gaussian(x,A,mu,sigma,B):
    return A*np.exp(-np.power(x-mu,2.0)/(2.0*sigma**2))+B

In [ ]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

# Data

Example how to load and convert files. Choose binning 1-8. Use binning 8 for high resolution

Load in dataset if you want to rotate by angle alpha

## (not used) AZ fitting: 1.5 K, 0 T, High E-resolution scan, (HHL)

In [ ]:
files = _tools.fileListGenerator("2644-2659",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHR = DataSet.DataSet(files)
dsHR.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
dsHR.mask = [np.logical_or(m1,m2) for m1,m2 in zip(dsHR.mask,np.sum(mask)(dsHR))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHR,treshold)
print(i)

## Global fitting: 1.6 K, 0 T, Global

### (HHL)

In [ ]:
files = _tools.fileListGenerator("2588-2608",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHHL = DataSet.DataSet(files)
dsHHL.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHHL,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHHL):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

### (H0L)

In [ ]:
files = _tools.fileListGenerator("1371-1388",r"U:\Data\CAMEA\MnF2\hdfH0L",2021)
dsH0L = DataSet.DataSet(files)
dsH0L.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsH0L,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsH0L):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L)) 

### (HK0)

In [ ]:
files = _tools.fileListGenerator("2745-2767",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHK0 = DataSet.DataSet(files)
dsHK0.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHK0,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHK0):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

### Update lattice parameters to restore expected symmetry of the dispersion

In [ ]:
dsHHL.updateSampleParameters([dsHHL[0].sample.a,dsHHL[0].sample.b,1.002*dsHHL[0].sample.c,dsHHL[0].sample.alpha,dsHHL[0].sample.beta,dsHHL[0].sample.gamma])
dsH0L.updateSampleParameters([0.992*dsH0L[0].sample.a,0.992*dsH0L[0].sample.b,0.987*dsH0L[0].sample.c,dsH0L[0].sample.alpha,dsH0L[0].sample.beta,dsH0L[0].sample.gamma])

## Other Scans

### 1.5 K, 10 T, Global scan, (HHL)

In [ ]:
files = _tools.fileListGenerator("2671-2694",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds10T = DataSet.DataSet(files)
ds10T.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds10T,treshold)
print(i)

# Symmetrize
for i,df in enumerate(ds10T):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

### 1.5 K, 10 T, ZFC, High E-resolution scan, (HHL)

In [ ]:
files = _tools.fileListGenerator("2695-2710",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds2 = DataSet.DataSet(files)
ds2.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds2.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds2.mask,np.sum(mask)(ds2))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds2,treshold)
print(i)

### 1.5 K, 10 T, FC, High E-resolution scan, (HHL)

In [ ]:
files = _tools.fileListGenerator("2711-2718",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
ds.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds.mask,np.sum(mask)(ds))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

### 1.5 K, 8 T, ZFC, (HK0)

In [ ]:
files = _tools.fileListGenerator("2783-2799",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds = DataSet.DataSet(files)
ds.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds,treshold)
print(i)

# Symmetrize
for i,df in enumerate(ds):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

# (not used) Masking

In case some data should be masked out (needs modification for your sample) (adapt accordingly)

In [ ]:
mask=[]
ds=ds
#Mask for Currate Axes spurions (Q points in [] and width given in dqx, dqy)
mask.append(Mask.CurratAxeMask([[1,1,0],[-1,-1,0],[1,0,0]],dqx=0.07,dqy=0.07))

ds.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds.mask,np.sum(mask)(ds))]

# Plotting

### 3D view of dataset

Binning along Qx, Qy, energy. Qx nd Qx = 0.02 are good if 1 degree steps in a3 are used, energy 0.1-0.2 is reasonable for overview

In [ ]:
%matplotlib qt
Viewer = dsHHL.View3D(0.03,0.03,.05,grid=9,rlu=True)
Viewer.caxis=(-0.0,3)
plt.show()

## Perform Cuts and Combine Datasets

### (not used) 1.5 K, 0 T, A to Z for J3

In [ ]:
dataFramesAZ = [] # Holder for data

QEListAZ = np.array([[0.5,0.5,0.5,6.4,7.5],
                [0.55,0.55,0.5,6.4,7.5],
                [0.6,0.6,0.5,6.4,7.5],
                [0.65,0.65,0.5,6.4,7.5],
                [0.7,0.7,0.5,6.4,7.5],
                [0.75,0.75,0.5,6.4,7.5],
                [0.8,0.8,0.5,6.4,7.5],
                [0.85,0.85,0.5,6.4,7.5],
                [0.9,0.9,0.5,6.4,7.5],
                [0.95,0.95,0.5,6.4,7.5],
                [1,1,0.5,6.4,7.5]])

widthAZ = 0.05 # 1/Å
minPixelAZ = 0.025 # meV

for QPoint in QEListAZ:
    dat,_ = dsHR.cut1DE(q=QPoint[0:3],E1=QPoint[3],E2=QPoint[4],width=widthAZ,minPixel=minPixelAZ,constantBins=True)
    dataFramesAZ.append(dat)

### 1.5 K, 0 T, Global

In [ ]:
dataFrames = [] # Holder for data

QEListHHL = np.array([[0,0,0.8,0.7,7.5],
                [0,0,0.85,0.7,7.5],
                [0,0,0.9,0.7,7.5],
                [0,0,0.95,0.7,7.5],
                [0,0,1,0.7,3.0],
                [0.5,0.5,0,5,6.8],
                [0.5,0.5,0.05,5,6.8],
                [0.5,0.5,0.1,5,6.9],
                [0.5,0.5,0.15,5,7.05],
                [0.5,0.5,0.2,5,7.2],
                [0.5,0.5,0.25,5,7.35],
                [0.5,0.5,0.3,5,7.5],
                [0.5,0.5,0.35,5,7.25],
                [0.5,0.5,0.4,5,7.5],
                [0.5,0.5,0.45,5,7.5],
                [0.5,0.5,0.5,5,7.5],
                [0.5,0.5,0.55,5,7.5],
                [0.5,0.5,0.6,5,7.5],
                [0.5,0.5,0.65,5,7.5],
                [0.5,0.5,0.7,5,7.5],
                [0.5,0.5,0.75,5,7.5],
                [0.5,0.5,0.8,5,7.5],
                [0.5,0.5,0.85,5,7.5],
                [0.5,0.5,0.9,5,7.5],
                [0.5,0.5,0.95,5,7.5],
                [0.5,0.5,1,5,7.5],
                [1,1,0.1,0.7,7.5],
                [1,1,0.15,0.7,7.5],
                [1,1,0.2,0.7,7.5],
                [1,1,0.25,0.7,7.5],
                [1,1,0.3,0.7,7.5],
                [1,1,0.35,0.7,7.5],
                [1,1,0.4,0.7,7.5],
                [1,1,0.45,0.7,7.5],
                [1,1,0.5,0.7,7.5],
                [1,1,0.55,0.7,7.5],
                [1,1,0.6,2.5,7.5],
                [1,1,0.65,2.5,7.5],
                [1,1,0.7,2.5,7.5],
                [1,1,0.75,2.5,7.5],
                [0.15,0.15,0.5,5,7.2],
                [0.2,0.2,0.5,5,7.5],
                [0.25,0.25,0.5,5,7.5],
                [0.3,0.3,0.5,5,7.5],
                [0.35,0.35,0.5,5,7.5],
                [0.4,0.4,0.5,5,7.5],
                [0.45,0.45,0.5,5,7.5],
#                [0.5,0.5,0.5,0.7,7.5],
                [0.55,0.55,0.5,0.7,7.5],
                [0.6,0.6,0.5,0.7,7.5],
                [0.65,0.65,0.5,0.7,7.5],
                [0.7,0.7,0.5,0.7,7.5],
                [0.75,0.75,0.5,0.7,7.5],
                [0.8,0.8,0.5,0.7,7.5],
                [0.85,0.85,0.5,0.7,7.5],
                [0.9,0.9,0.5,0.7,7.5],
                [0.95,0.95,0.5,0.7,7.5],
#                [1,1,0.5,0.7,7.5],
                [0.05,0.05,1,0.7,7.5],
                [0.1,0.1,1,0.7,7.5],
                [0.15,0.15,1,0.7,7.5],
                [0.2,0.2,1,0.7,7.5],
                [0.25,0.25,1,0.7,7.5],
                [0.3,0.3,1,0.7,7.5],
                [0.35,0.35,1,0.7,7.5],
                [0.4,0.4,1,0.7,7.5],
                [0.45,0.45,1,0.7,7.5],
#                [0.5,0.5,1,0.7,7.5],
                [0.55,0.55,1,0.7,7.5],
                [0.6,0.6,1,0.7,7.5],
                ])

QEListH0L = np.array([[0.7,0,0,0.7,6],
                [0.75,0,0,0.7,6],
                [0.8,0,0,0.7,6.5],
                [0.85,0,0,0.7,7],
                [0.9,0,0,0.9,5],
                [0.95,0,0,0.6,4],
                [1,0,0,0.6,4],
                [1.05,0,0,0.6,4],
                [1.1,0,0,0.9,5],
                [1.15,0,0,1.8,7],
                [1.2,0,0,2.7,7],
                [1.25,0,0,2.7,7],
                [1.3,0,0,2.7,7],
                [1.35,0,0,2.7,7],
                [1.4,0,0,2.7,7],
                [1.45,0,0,3.5,7],
                [1.5,0,0,3.5,7],
                [0.7,0,0.5,5,7.4],
                [0.75,0,0.5,5,7.4],
                [0.8,0,0.5,5,7.4],
                [0.85,0,0.5,5,7.4],
                [0.9,0,0.5,5,7.4],
                [0.95,0,0.5,5,7.4],
                [1,0,0.5,5,7.4],
                [1.05,0,0.5,5,7.4],
                [1.1,0,0.5,5,7.4],
                [1.15,0,0.5,5,7.4],
                [1.2,0,0.5,5,7.4],
                [1.25,0,0.5,5,7.4],
                [1.3,0,0.5,5,7.4],
                [1.35,0,0.5,5,7.4],
                [1.4,0,0.5,5,7.4],
                [1.45,0,0.5,5,7.4],
                [1.5,0,0.5,6.4,7.4],
                [1,0,0.2,1,7.4],
                [1,0,0.25,2,7.4],
                [1,0,0.3,3,7.4],
                [1,0,0.35,3.5,7.4],
                [1,0,0.4,4,7.4],
                [1,0,0.45,4,7.4],
                [1,0,0.5,4,7.4],
                [1,0,0.55,4,7.4],
                [1,0,0.6,4,7.4],
                [1,0,0.65,3.5,7.4],
                [1,0,0.7,3.5,7.4],
                [1.5,0,0.05,5.5,7.4],
                [1.5,0,0.1,5.5,7.4],
                [1.5,0,0.15,5.5,7.4],
                [1.5,0,0.2,5.5,7.4],
                [1.5,0,0.25,5.5,7.4],
                [1.5,0,0.3,5.5,7.4],
                [1.5,0,0.35,5.5,7.4],
                [1.5,0,0.4,5.5,7.4],
                [1.5,0,0.45,6.4,7.4],
                [1.5,0,0.5,6.4,7.4]
                ])

QEListHK0 = np.array([[0.55,0.5,0,5,7.4],
                [0.6,0.5,0,5,7.4],
                [0.65,0.5,0,5,7.4],
                [0.7,0.5,0,5,7.4],
                [0.75,0.5,0,5,7.4],
                [0.8,0.5,0,5,7.4],
                [0.85,0.5,0,5,7.4],
                [0.9,0.5,0,5,7.4],
                [0.95,0.5,0,5,7.4],
                [1,0.5,0,5,7.4],
                [1.05,0.5,0,5,7.4],
                [1.1,0.5,0,5,7.4],
                [1.15,0.5,0,5,7.4],
                [1.2,0.5,0,5,7.4],
                [1.25,0.5,0,5,7.4],
                [1.3,0.5,0,5,7.4],
                [1.35,0.5,0,5,7.4],
                [1.4,0.5,0,5,7.4],
                [1.45,0.5,0,5,7.4],
                [1.5,0.5,0,5,7.4]
                ])

widthHHL = 0.03 # 1/Å
minPixelHHL = 0.05 # meV

widthH0L = 0.03 # 1/Å
minPixelH0L = 0.05 # meV

widthHK0 = 0.03 # 1/Å
minPixelHK0 = 0.05 # meV

for QPoint in QEListHHL:
    dat,_ = dsHHL.cut1DE(q=QPoint[0:3],E1=QPoint[3],E2=QPoint[4],width=widthHHL,minPixel=minPixelHHL,constantBins=True)
    dataFrames.append(dat)

for QPoint in QEListH0L:
    dat,_ = dsH0L.cut1DE(q=QPoint[0:3],E1=QPoint[3],E2=QPoint[4],width=widthH0L,minPixel=minPixelH0L,constantBins=True)
    dataFrames.append(dat)

for QPoint in QEListHK0:
    dat,_ = dsHK0.cut1DE(q=QPoint[0:3],E1=QPoint[3],E2=QPoint[4],width=widthHK0,minPixel=minPixelHK0,constantBins=True)
    dataFrames.append(dat)

%matplotlib inline

#combiFig, combiAxis = plt.subplots()

#for d in dataFrames:
#    Intensity = d['Int']
    # The error is calculated using the Gaussian approximation and normalized to monitor and instrument sensitivity
#    Error = d['Int_err']
#    E = d['Energy']
#    combiAxis.errorbar(E,Intensity,yerr=Error,fmt='.-', label='({:.2f}, {:.2f}, {:.2f})'.format(np.mean(d['H']), np.mean(d['K']), np.mean(d['L'])))
    
#combiAxis.set_xlabel('E [meV]')
#combiAxis.set_ylabel('Intensity [arb]')
#combiAxis.legend(frameon=False)
#combiAxis.get_figure().set_size_inches(16,12)
#combiFig

## Fit the peaks and output the text file

### (not used) Fit the peaks and output the text file for SpinW: 0 T A to Z for J3

In [ ]:
%matplotlib inline

from scipy.optimize import curve_fit
fitE = []
errE = []
fitI = []
errI = []
for i in range(len(dataFramesAZ)):
    fitFigAZ,fitAxAZ = plt.subplots()

    IntensityPre = dataFramesAZ[i]['Int']
    ErrorPre = dataFramesAZ[i]['Int_err']
    EPre = dataFramesAZ[i]['Energy']

    # Drop bins with 0 counts, reset Pandas series indexing
    Intensity = IntensityPre[IntensityPre!=0].reset_index(drop=True)
    Error = ErrorPre[IntensityPre!=0].reset_index(drop=True)
    E = EPre[IntensityPre!=0].reset_index(drop=True)

    # The guess follows the input list from the function. Amplitude and center position is found from the maximal intensity position
    centerId = np.argmax(Intensity) # Index of maximal positin
    mu_guess = E[centerId]
    A_guess = Intensity[centerId]
    sigma_guess = 0.1 # Pure gut feel guess
    B_guess = np.min(Intensity)
    guess = [A_guess,mu_guess,sigma_guess,B_guess]

    # Call the optimization function, which returns the optimal values (fit) and the correlation 
    # matrix (Var_mat) from which the actual error estiamte can be found
    fit,Var_mat = curve_fit(Gaussian,E,Intensity,p0=guess,sigma=Error)

    # For simplicity, it is assumed that the errors are uncorrelated and thus equal to the square root of the diagonal
    fit_err = np.sqrt(np.diag(Var_mat))

    # Create a variable against which the fit is plotted
    X = np.linspace(np.min(E),np.max(E),101)
    Y = Gaussian(X,*fit) # Notice: using the * operator here unpacks all of the values so that you don't have to write them by hand
    Init = Gaussian(X,*guess)

    # Plot data
    fitAxAZ.errorbar(E,Intensity,yerr=Error,fmt='.-', label='({:.2f}, {:.2f}, {:.2f})'.format(np.mean(dataFramesAZ[i]['H']), np.mean(dataFramesAZ[i]['K']), np.mean(dataFramesAZ[i]['L'])))
    # Plot fit
    fitAxAZ.plot(X,Y,label='Fit')
    fitAxAZ.plot(X,Init,label='Initial Guess')
    fitAxAZ.set_xlabel('E [meV]')
    fitAxAZ.set_ylabel('Intensity [Arb]')
    fitAxAZ.legend()
    fitE.append(fit[1])
    errE.append(fit_err[1]) # Square root of diagonal of covariance matrix
    fitI.append(fit[0])
    errI.append(fit_err[0])
    plt.show()
    print('Q: ({:.2f}, {:.2f}, {:.2f}). E = {:.3f} +- {:.3f}'.format(np.mean(dataFramesAZ[i]['H']), np.mean(dataFramesAZ[i]['K']), np.mean(dataFramesAZ[i]['L']), fit[1], fit_err[1]))

Save fitted dispersion to text file for SpinW

In [ ]:
txtFilename = '1p5K_0T_AZ_FitDisp'

txtFile = 'U:/Data/MATLAB/MnF2/' + txtFilename + '.txt'
fid = open(txtFile, 'w')
fid.write('QH QK QL ENlim1 ENlim2 I1 EN1 s1\n')
for i in range(len(dataFramesAZ)):
    d = dataFramesAZ[i]
    txt = "{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}\n"
    fid.write(txt.format(np.mean(d['H']), np.mean(d['K']), np.mean(d['L']), 0.5, 10, fitI[i], fitE[i], errE[i]))

fid.close()

### Fit the peaks and output the text file for SpinW: 0 T Global

Fit The Peaks

In [ ]:
%matplotlib inline
# Run multiple times for different dataframes or make it into a loop

from scipy.optimize import curve_fit
fitE = []
errE = []
fitI = []
errI = []
for i in range(len(dataFrames)):
    fitFig,fitAx = plt.subplots()

    IntensityPre = dataFrames[i]['Int']
    ErrorPre = dataFrames[i]['Int_err']
    EPre = dataFrames[i]['Energy']

    # Drop bins with 0 counts, reset Pandas series indexing
    Intensity = IntensityPre[IntensityPre!=0].reset_index(drop=True)
    Error = ErrorPre[IntensityPre!=0].reset_index(drop=True)
    E = EPre[IntensityPre!=0].reset_index(drop=True)

    # The guess follows the input list from the function. Amplitude and center position is found from the maximal intensity position
    centerId = np.argmax(Intensity) # Index of maximal positin
    mu_guess = E[centerId]
    A_guess = Intensity[centerId]
    sigma_guess = 0.25 # Pure gut feel guess
    B_guess = np.min(Intensity)
    guess = [A_guess,mu_guess,sigma_guess,B_guess]

    # Call the optimization function, which returns the optimal values (fit) and the correlation 
    # matrix (Var_mat) from which the actual error estiamte can be found
    fit,Var_mat = curve_fit(Gaussian,E,Intensity,p0=guess,sigma=Error)

    # For simplicity, it is assumed that the errors are uncorrelated and thus equal to the square root of the diagonal
    fit_err = np.sqrt(np.diag(Var_mat))

    # Create a variable against which the fit is plotted
    X = np.linspace(np.min(E),np.max(E),101)
    Y = Gaussian(X,*fit) # Notice: using the * operator here unpacks all of the values so that you don't have to write them by hand
    Init = Gaussian(X,*guess)

    # Plot data
    fitAx.errorbar(E,Intensity,yerr=Error,fmt='.-', label='({:.2f}, {:.2f}, {:.2f})'.format(np.mean(dataFrames[i]['H']), np.mean(dataFrames[i]['K']), np.mean(dataFrames[i]['L'])))
    # Plot fit
    fitAx.plot(X,Y,label='Fit')
    fitAx.plot(X,Init,label='Initial Guess')
    fitAx.set_xlabel('E [meV]')
    fitAx.set_ylabel('Intensity [Arb]')
    fitAx.legend()
    #if i==4 or i==73: # Use Johnson 1959 AFMR result as discussed in Hagiwara 1996
    #    fitE.append(1.081) # Unit conversion: https://www.ill.eu/fileadmin/user_upload/ILL/3_Users/Support_labs_infrastructure/Software-tools/DIF_tools/neutrons.html
    #    errE.append(0.006)
    #else:
    #    fitE.append(fit[1])
    #    errE.append(fit_err[1]) # Square root of diagonal of covariance matrix
    fitE.append(fit[1])
    errE.append(fit_err[1]) # Square root of diagonal of covariance matrix
    fitI.append(fit[0])
    errI.append(fit_err[0])
    plt.show()
    print('Q: ({:.2f}, {:.2f}, {:.2f}). E = {:.3f} +- {:.3f}'.format(np.mean(dataFrames[i]['H']), np.mean(dataFrames[i]['K']), np.mean(dataFrames[i]['L']), fit[1], fit_err[1]))

In [ ]:
# Save fitted dispersion to text file for SpinW
xtFilename = '1p5K_0T_FitDisp'

txtFile = 'U:/Data/MATLAB/MnF2/' + txtFilename + '.txt'
fid = open(txtFile, 'w')
fid.write('QH QK QL ENlim1 ENlim2 I1 EN1 s1\n')
for i in range(len(dataFrames)):
    d = dataFrames[i]
    txt = "{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}\n"
    fid.write(txt.format(np.mean(d['H']), np.mean(d['K']), np.mean(d['L']), 0.5, 10, fitI[i], fitE[i], errE[i]))

fid.close()